In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv(r'/content/drive/My Drive/Deep Learning/ECG/ECG-Dataset/ECG.csv')
dataset_500_signal = dataset[0:62500]

X = dataset_500_signal.iloc[:, 1:-1]
Y = dataset_500_signal.iloc[:, -1]

from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X = sc_X.fit_transform(X)
Y = sc_y.fit_transform(np.array(Y).reshape(-1,1))

X =np.array(X).reshape(62500, 1, 1)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [4]:
  # create model
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(1, 1)))
model.add(LSTM(110, activation='relu', return_sequences=True))
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(1, 1)))
model.add(Dense(4, activation='softmax'))
	# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 128)            66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 110)            105160    
_________________________________________________________________
bidirectional (Bidirectional (None, 100)               64400     
_________________________________________________________________
dense (Dense)                (None, 4)                 404       
Total params: 236,524
Trainable params: 236,524
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
model.fit(X_train, y_train, epochs=100, batch_size=5)

Epoch 1/100
10000/10000 [==============================] - 36s 4ms/step - loss: 0.5263 - accuracy: 0.5260
Epoch 2/100
10000/10000 [==============================] - 36s 4ms/step - loss: 0.4774 - accuracy: 0.5702
Epoch 3/100
10000/10000 [==============================] - 36s 4ms/step - loss: 0.4728 - accuracy: 0.5710
Epoch 4/100
10000/10000 [==============================] - 35s 4ms/step - loss: 0.4679 - accuracy: 0.5712
Epoch 5/100
10000/10000 [==============================] - 36s 4ms/step - loss: 0.4670 - accuracy: 0.5720
Epoch 6/100
10000/10000 [==============================] - 34s 3ms/step - loss: 0.4666 - accuracy: 0.5709
Epoch 7/100
10000/10000 [==============================] - 35s 3ms/step - loss: 0.4657 - accuracy: 0.5712
Epoch 8/100
10000/10000 [==============================] - 39s 4ms/step - loss: 0.4656 - accuracy: 0.5721
Epoch 9/100
10000/10000 [==============================] - 47s 5ms/step - loss: 0.4652 - accuracy: 0.5722
Epoch 10/100
10000/10000 [====================

In [6]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 58.54%


# **Cross Validation**

In [7]:
dataset = pd.read_csv(r'/content/drive/My Drive/Deep Learning/ECG/Dataset/ecg1.csv')

X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]
X = X.values.reshape(1250, 1, 1)

In [8]:
# define baseline model
def baseline_model():
	# create model
  model = Sequential()
  model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(1, 1)))
  model.add(Dense(1))
  model.add(Dense(4, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [9]:
from sklearn.model_selection import cross_val_score

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Accuracy: %.3f%%" % (results.mean()*100))
print("Starndard daviation: %.3f%%" % (results.std()*100))

Accuracy: 61.280%
Starndard daviation: 4.035%
